<a href="https://colab.research.google.com/github/IsraelBosun/stock-prediction/blob/main/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install yfinance gnews torch tqdm vaderSentiment ta -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 12.5 MB/s eta 0:00:00


In [3]:
import warnings
warnings.filterwarnings("ignore")

**<h1>Loading The Dataset </h1>**

In [5]:
import yfinance as yf
import pandas as pd

# Define top 6 U.S. stocks (including Tesla)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "NVDA", "TSLA"]

# Download all data
download = yf.download(tickers, start="2020-01-01", end="2025-10-26", group_by="ticker")

# Stack so all tickers appear under each other
df = (
    download.stack(level=0)  # Moves ticker from column to row
    .rename_axis(["Date", "Ticker"])  # Renames the new row index
    .reset_index()  # Converts back to normal DataFrame
)

# Show result
df.head()

[*********************100%***********************]  6 of 6 completed


Price,Date,Ticker,Open,High,Low,Close,Volume
0,2020-01-02,AAPL,71.545882,72.598884,71.292296,72.538506,135480400
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,80580000
2,2020-01-02,GOOGL,66.958671,67.965225,66.863328,67.965225,27278000
3,2020-01-02,MSFT,151.040795,152.895746,150.612732,152.791107,22622100
4,2020-01-02,NVDA,5.942537,5.971410,5.892010,5.971410,237536000


In [6]:
from google.colab import files
import os

# Save the DataFrame to a CSV file
csv_file_path = "stock_data.csv"
download.to_csv(csv_file_path)

# Download the CSV file
files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Calculate Technical Indicators

In [7]:
import pandas as pd
import ta  # pip install ta

# Assuming df has columns: Open, High, Low, Close, Volume
df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
df['macd'] = ta.trend.MACD(df['Close']).macd()
df['atr'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close']).average_true_range()
df['ema_20'] = ta.trend.EMAIndicator(df['Close'], window=20).ema_indicator()
df['bollinger_w'] = ta.volatility.BollingerBands(df['Close']).bollinger_wband()


df = df.dropna().reset_index(drop =True)

df.head()

Price,Date,Ticker,Open,High,Low,Close,Volume,rsi,macd,atr,ema_20,bollinger_w
0,2020-01-08,AMZN,94.902000,95.550003,94.321999,94.598503,70160000,51.978120,-0.449769,53.852603,70.586206,246.096547
1,2020-01-08,GOOGL,69.263281,70.108950,69.154534,69.770782,35314000,49.912338,-0.464911,51.823415,70.508547,246.462608
2,2020-01-08,MSFT,151.183463,152.962326,150.251234,152.286926,27746500,56.149980,6.111008,54.063996,78.296964,253.905673
3,2020-01-08,NVDA,5.967677,6.024427,5.927604,5.983109,277108000,45.363140,-0.477520,60.656519,71.409930,277.730139
4,2020-01-08,TSLA,31.580000,33.232666,31.215334,32.809334,467164500,47.360007,-3.494046,58.270307,67.733683,285.340092


### Initialize News and Sentiment Tools

In [8]:
from gnews import GNews
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Google News API wrapper
news = GNews(language='en', country='US', period='1d', max_results=5)

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


### Fetch and Process Sentiment Data

In [9]:
import os
import json
import pandas as pd
import numpy as np
import asyncio
import aiohttp
from datetime import datetime, timedelta
from gnews import GNews
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

CACHE_FILE = "/content/drive/MyDrive/Python Projects/Mathematics/sentiment_cache.json"

# Load cache if exists
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        sentiment_cache = json.load(f)
else:
    sentiment_cache = {}

analyzer = SentimentIntensityAnalyzer()

# --- Async Sentiment Function ---
async def fetch_sentiment(session, ticker, date):
    key = f"{ticker}_{date}"
    if key in sentiment_cache:
        return key, sentiment_cache[key]

    try:
        # ✅ Handle both “YYYY-MM-DD” and “YYYY-MM-DD HH:MM:SS”
        if isinstance(date, pd.Timestamp):
            start = date.to_pydatetime()
        else:
            try:
                start = datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S")
            except ValueError:
                start = datetime.strptime(str(date), "%Y-%m-%d")

        end = start + timedelta(days=1)

        # Query Google News for the specific day
        news = GNews(language='en', country='US', max_results=5,
                     start_date=start, end_date=end)
        query = f"{ticker} stock"
        results = news.get_news(query)

        sentiments = []
        for r in results:
            title = r.get("title", "")
            if not title:
                continue
            score = analyzer.polarity_scores(title)["compound"]
            sentiments.append(score)

        avg_score = float(np.mean(sentiments)) if sentiments else 0.0
        sentiment_cache[key] = avg_score
        print(f"📅 {date} | {ticker} | 📰 {len(sentiments)} | Avg: {avg_score:+.3f}")
        return key, avg_score

    except Exception as e:
        print(f"⚠️ Error for {ticker} on {date}: {e}")
        return key, 0.0


# --- Run in Async Batches for Speed ---
async def process_all(df):
    sentiments = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        # Use tqdm with pandas apply for progress bar
        for i, row in tqdm(df.iterrows(), total=len(df), desc="Fetching sentiment"):
            ticker = row['Ticker']
            date = row['Date']
            tasks.append(fetch_sentiment(session, ticker, date))

            # Run in manageable batches
            if len(tasks) % 200 == 0 or i == len(df) - 1:
                results = await asyncio.gather(*tasks)
                for key, score in results:
                    sentiments.append(score)

                # 🧠 Save cache periodically
                with open(CACHE_FILE, "w") as f:
                    json.dump(sentiment_cache, f)
                print(f"💾 Cache saved ({len(sentiment_cache)} entries)")
                tasks = []  # reset batch

    return sentiments


sentiment_scores = await process_all(df) # Use await to run in the existing loop
df['sentiment_score'] = sentiment_scores

# ✅ Save final DataFrame
output_path = "/content/drive/MyDrive/Python Projects/Mathematics/final_with_sentiment.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ All done! File saved to: {output_path}")

Fetching sentiment:   6%|▋         | 563/8747 [00:00<00:03, 2468.77it/s]

💾 Cache saved (8747 entries)
💾 Cache saved (8747 entries)


Fetching sentiment:  12%|█▏        | 1065/8747 [00:00<00:04, 1833.98it/s]

💾 Cache saved (8747 entries)
💾 Cache saved (8747 entries)
💾 Cache saved (8747 entries)


Fetching sentiment:  18%|█▊        | 1600/8747 [00:00<00:03, 2100.54it/s]

...

💾 Cache saved (8747 entries)
💾 Cache saved (8747 entries)
💾 Cache saved (8747 entries)


Fetching sentiment:  99%|█████████▉| 8664/8747 [00:04<00:00, 1887.15it/s]

💾 Cache saved (8747 entries)
💾 Cache saved (8747 entries)


Fetching sentiment: 100%|██████████| 8747/8747 [00:05<00:00, 1720.55it/s]


💾 Cache saved (8747 entries)

✅ All done! File saved to: /content/drive/MyDrive/Python Projects/Mathematics/final_with_sentiment.csv


In [10]:
df.head()

Price,Date,Ticker,Open,High,Low,Close,Volume,rsi,macd,atr,ema_20,bollinger_w,sentiment_score
0,2020-01-08,AMZN,94.902000,95.550003,94.321999,94.598503,70160000,51.978120,-0.449769,53.852603,70.586206,246.096547,0.28928
1,2020-01-08,GOOGL,69.263281,70.108950,69.154534,69.770782,35314000,49.912338,-0.464911,51.823415,70.508547,246.462608,-0.13660
2,2020-01-08,MSFT,151.183463,152.962326,150.251234,152.286926,27746500,56.149980,6.111008,54.063996,78.296964,253.905673,0.37740
3,2020-01-08,NVDA,5.967677,6.024427,5.927604,5.983109,277108000,45.363140,-0.477520,60.656519,71.409930,277.730139,-0.15572
4,2020-01-08,TSLA,31.580000,33.232666,31.215334,32.809334,467164500,47.360007,-3.494046,58.270307,67.733683,285.340092,0.08882


### Visualize Close vs Sentiment Score

In [12]:
import plotly.express as px

# Get all unique tickers
tickers = df['Ticker'].unique()

# Loop through each ticker and plot scatter + regression line
for ticker in tickers:
    ticker_df = df[df["Ticker"] == ticker].copy()

    fig = px.scatter(
        ticker_df,
        x="Close",
        y="sentiment_score",
        trendline="ols",
        title=f"{ticker} — Close vs Sentiment Score",
        labels={"Close": "Closing Price", "sentiment_score": "Sentiment Score"}
    )
    fig.show()

In [13]:
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.head(10)

Price,Date,Ticker,Open,High,Low,Close,Volume,rsi,macd,atr,ema_20,bollinger_w,sentiment_score,Target
0,2020-01-08,AMZN,94.902000,95.550003,94.321999,94.598503,70160000,51.978120,-0.449769,53.852603,70.586206,246.096547,0.28928,0
1,2020-01-08,GOOGL,69.263281,70.108950,69.154534,69.770782,35314000,49.912338,-0.464911,51.823415,70.508547,246.462608,-0.13660,1
2,2020-01-08,MSFT,151.183463,152.962326,150.251234,152.286926,27746500,56.149980,6.111008,54.063996,78.296964,253.905673,0.37740,0
3,2020-01-08,NVDA,5.967677,6.024427,5.927604,5.983109,277108000,45.363140,-0.477520,60.656519,71.409930,277.730139,-0.15572,1
4,2020-01-08,TSLA,31.580000,33.232666,31.215334,32.809334,467164500,47.360007,-3.494046,58.270307,67.733683,285.340092,0.08882,1
5,2020-01-09,AAPL,74.202542,74.972970,73.951373,74.779762,170108400,50.413489,-2.469535,57.119831,68.404738,259.334049,0.22940,1
6,2020-01-09,AMZN,95.494499,95.890999,94.790001,95.052498,63346000,51.865889,-0.021514,54.547788,70.942620,244.166356,0.11230,0
7,2020-01-09,GOOGL,70.609497,70.944686,70.027512,70.503235,33200000,49.957639,-0.061650,52.439017,70.900774,244.497756,0.19044,1
8,2020-01-09,MSFT,153.951654,154.313137,153.181137,154.189468,21385000,55.912470,6.583432,54.679794,78.833030,252.374169,0.45880,0
9,2020-01-09,NVDA,6.069477,6.121248,5.995055,6.048818,255112000,45.574049,-0.102819,61.359410,71.901201,276.462583,0.00000,1


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Feature Selection for Training

In [15]:
from sklearn.model_selection import train_test_split

features = ['Open', 'High', 'Low', 'Close', 'Volume', 'rsi', 'macd', 'atr', 'ema_20', 'bollinger_w', 'sentiment_score']
X = df[features]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)

# scaling

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Train Logistic Regression Model

In [16]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=42)

### Evaluate Model Performance

In [17]:
y_pred = model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8908571428571429

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.95      0.91      1040
           1       0.91      0.81      0.86       710

    accuracy                           0.89      1750
   macro avg       0.90      0.88      0.88      1750
weighted avg       0.89      0.89      0.89      1750


Confusion Matrix:
 [[986  54]
 [137 573]]


### Display Feature Importance

In [18]:
importance = pd.DataFrame({
    'Feature': features,
    'Coefficient': model.coef_[0]
}).sort_values('Coefficient', ascending=False)

print(importance)

            Feature  Coefficient
4            Volume     0.370136
8            ema_20     0.250630
7               atr    -0.194498
2               Low    -0.320250
9       bollinger_w    -0.431829
10  sentiment_score    -0.545639
3             Close    -0.953797
6              macd    -1.099633
0              Open    -1.229153
1              High    -1.770562
5               rsi    -3.340796


# **Inference**

### Define Prediction Function

In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
from gnews import GNews
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def predict_ticker_direction(ticker):
    # 1️⃣ Download market data
    downloads = yf.download(ticker, period="60d", group_by="ticker", progress = False)

    data = (
        downloads.stack(level=0, future_stack = True)
        .rename_axis(["Date", "Ticker"])
        .reset_index()
    )

    # 2️⃣ Technical Indicators
    data['rsi'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()
    data['macd'] = ta.trend.MACD(data['Close']).macd()
    data['atr'] = ta.volatility.AverageTrueRange(data['High'], data['Low'], data['Close']).average_true_range()
    data['ema_20'] = ta.trend.EMAIndicator(data['Close'], window=20).ema_indicator()
    data['bollinger_w'] = ta.volatility.BollingerBands(data['Close']).bollinger_wband()

    data = data.dropna()

    # 3️⃣ Sentiment from news
    news = GNews(language='en', country='US', period='1d', max_results=5)
    analyzer = SentimentIntensityAnalyzer()

    results = news.get_news(f"{ticker} stock")
    sentiments = [analyzer.polarity_scores(r['title'])['compound'] for r in results]
    sentiment_today = np.mean(sentiments) if sentiments else 0

    # 4️⃣ Last candle's features
    latest = data.iloc[-1].copy()
    latest['sentiment_score'] = sentiment_today

    features = ['Open','High','Low','Close','Volume',
                'rsi','macd','atr','ema_20','bollinger_w','sentiment_score']

    X_new = pd.DataFrame([latest[features].to_dict()])

    # 5️⃣ Scale and predict
    X_new_scaled = scaler.transform(X_new)
    prediction = model.predict(X_new_scaled)[0]
    prob = model.predict_proba(X_new_scaled)[0][1]

    return {
        "ticker": ticker,
        "prediction": "UP 📈" if prediction == 1 else "DOWN 📉",
        "up_probability": round(prob, 3),
        "sentiment": round(sentiment_today, 3)
    }


### Predict Direction for Multiple Tickers

In [20]:
# tickers = ["AAPL", "TSLA", "GOOGL", "MSFT", "AMZN"]
tickers = [
    "AAPL","TSLA","GOOGL","MSFT","AMZN",
    "META","NVDA","NFLX","AMD","INTC",
    "JPM","BAC","WFC","GS","MS",
    "XOM","CVX","COP","BP","SLB",
    "JNJ","PFE","MRNA","UNH","ABBV",
    "WMT","COST","TGT","NKE","SBUX",
    "RIVN","LCID","F","GM"
]



results = []

for t in tickers:
    try:
        res = predict_ticker_direction(t)
        results.append(res)
        print(res)
    except Exception as e:
        print(f"Error with {t}: {e}")


{'ticker': 'AAPL', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.0), 'sentiment': np.float64(0.391)}
{'ticker': 'TSLA', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.0), 'sentiment': np.float64(0.0)}
{'ticker': 'GOOGL', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.0), 'sentiment': np.float64(0.351)}
{'ticker': 'MSFT', 'prediction': 'UP 📈', 'up_probability': np.float64(0.961), 'sentiment': np.float64(0.235)}
{'ticker': 'AMZN', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.0), 'sentiment': np.float64(0.322)}
{'ticker': 'META', 'prediction': 'UP 📈', 'up_probability': np.float64(1.0), 'sentiment': np.float64(-0.195)}
{'ticker': 'NVDA', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.003), 'sentiment': np.float64(-0.13)}
{'ticker': 'NFLX', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.0), 'sentiment': np.float64(0.215)}
{'ticker': 'AMD', 'prediction': 'DOWN 📉', 'up_probability': np.float64(0.0), 'sentiment': np.float64(-0.038)}
{'ticker':